In [ ]:
! ls /media/bigdata/s4431520/data/preprocessed_corpus/compiled.csv/

In [1]:
def save_and_get_authors_df(df, outdir):
    authors = []
    for v in df.values: 
        i = v[3]
        string_dict = i.replace("'", '"')
        try: 
            author_dict = json.loads(string_dict)
            for a in author_dict: 
                a["paper"] = v[2]
            authors.extend(author_dict)
        except: # since i replace ' with ", this impaces names with ' in them
            pass

    author_df = pd.DataFrame.from_dict(authors)
    author_df = author_df.dropna() # probably unnecessary 
    
    print(author_df.head())
    
    # Convert id list to string and remove duplicates 
    author_df["ids"] = author_df.ids.astype(str)    
    author_df = author_df[~author_df.duplicated()]
    df = df.reset_index(drop=True)
    
    author_df.to_csv(outdir+'authors.csv')
    return author_df

In [2]:
import pandas as pd 
import dask as dd 
import sys
import csv
from sklearn.model_selection import train_test_split
import glob 
from data_loader import * 

In [4]:
csv.field_size_limit(sys.maxsize)

datadir = "/media/bigdata/s4431520/data/preprocessed_corpus/compiled.csv/"

In [ ]:
# df = pd.read_csv(datadir, compression='gzip')
# df.to_parquet(datadir+"compiled.parquet", compression='gzip')

files = sorted(glob.glob(datadir+"*.part"))
dfs = [pd.read_csv(p, compression="gzip") for p in files]
df = pd.concat(dfs)

In [ ]:
# more preprocessing 
df = df[COLUMNS]
df = df.dropna()
df = df.drop(['entities'], axis=1)

# Parse datatypes 
# df = df.reset_index(drop=True)
# df.year = pd.to_numeric(df.year, downcast='signed')
# df.outCitations = [i.strip("[]'").split("', '") for i in df.outCitations]
# df.inCitations = [i.strip("[]'").split("', '") for i in df.inCitations]

In [ ]:
adf = save_and_get_authors_df(df, "/media/bigdata/s4431520/data/preprocessed_corpus/")

In [14]:
# remove nan entities rows 
# df.to_csv("/media/bigdata/s4431520/data/preprocessed_corpus/compiled_full.csv", compression='gzip')
# df.to_parquet("/media/bigdata/s4431520/data/preprocessed_corpus/compiled_full.parquet", engine='fastparquet', compression='gzip')

In [3]:
df = pd.read_csv("/media/bigdata/s4431520/data/preprocessed_corpus/compiled_full.csv", compression='gzip', index_col=0)

/home/s4431520/.local/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
adf = save_and_get_authors_df(df, "/media/bigdata/s4431520/data/preprocessed_corpus/")

                           name           ids
0                    Ka-Ho Wong     [2035199]
1                    Wai Kit Lo    [31726096]
2                 Helen M. Meng   [145199941]
3  Johannes Schulz-Stellenfleth  [1398434700]
4                Susanne Lehner    [51226504]


In [12]:
# # Split train:test:val 70:15:15
# df_train, df_test = train_test_split(df, train_size=0.7, test_size=0.3, random_state=420)
# df_test, df_val = train_test_split(df_test, train_size=0.5, test_size=0.5, random_state=420)

# print(df_train.shape, df_test.shape, df_val.shape)

# df_train.to_csv("train.csv", compression='gzip')
# df_test.to_csv("test.csv", compression='gzip')
# df_val.to_csv("val.csv", compression='gzip')

(1833048, 10) (392796, 10) (392796, 10)


In [6]:
adf
# [{name:a_name, ids:id_list, papers: paper_list}]

,name,ids
0,Ka-Ho Wong,['2035199']
1,Wai Kit Lo,['31726096']
2,Helen M. Meng,['145199941']
3,Johannes Schulz-Stellenfleth,['1398434700']
4,Susanne Lehner,['51226504']
...,...,...
7949161,Ji Hyuk Han,['5505255']
7949164,Giles Blaney,['51904482']
7949166,Sergio Fantini,['47330835']
7949177,Davut Deniz Yavuz,['153085030']


In [13]:
df_train.head()

,year,outCitations,id,authors,journalName,paperAbstract,inCitations,title,doi,venue
217992,1992,"['c0cb52d2aefecc951cbcbf1dad796718ba7d69b5', '...",79a05cbd919d629e07a86c639e612528656eb4cc,"[{'name': 'Guangjun Liu', 'ids': ['1723284']},...",Proceedings 1992 IEEE International Conference...,Several robust control schemes and a computed ...,"['5ff638f03d54a386a691a6da7e2ae6c8440ac4e0', '...",Experiments on robust control of robot manipul...,10.1109/ROBOT.1992.219946,Proceedings 1992 IEEE International Conference...
1655773,2015,[''],9de14a4bd2e77ddf7939764d776218b88de21835,"[{'name': 'Mario Lehenbauer-Baum', 'ids': ['14...",Computers in Human Behavior,"93.6% play on level 85 in World of Warcraft, o...","['77e2690ad883481e3026115c3f7975f3e1f03937', '...",Towards classification criteria for internet g...,10.1016/j.chb.2014.11.098,Computers in Human Behavior
2433426,2013,[''],898284d126bf4d1b3f5c8820a5b46055c8628a81,"[{'name': 'Yahya Forghani', 'ids': ['2636007']...",Expert Systems,Data may be afflicted with uncertainty. Uncert...,"['d3f32082e3feadb22551af52a958225f7accab0d', '...",Classification of fuzzy data based on the supp...,10.1111/j.1468-0394.2012.00645.x,Expert Systems
10492,2007,"['123ef85e0ea0b3d36dc1769614fcd30e1c32a6d0', '...",719f64781591ed4b10f1351e8241e814c09259b8,"[{'name': 'Serhan Yarkan', 'ids': ['47093417']...",2007 IEEE 66th Vehicular Technology Conference,One of the major goals of cognitive radio (CR)...,"['3f6cf539f76b1f4066a5a2600cb25d3b32096ac3', '...",Binary Time Series Approach to Spectrum Predic...,10.1109/VETECF.2007.332,2007 IEEE 66th Vehicular Technology Conference
2370939,1998,[''],18d63aa0084dc4704c809e68540a3521a166d5d4,"[{'name': 'Olga Volkoff', 'ids': ['2706472']},...",Journal of Information Technology,With changes in both technology and the enviro...,[''],Nortel—reinventing IS,10.1080/026839699344575,JIT


In [17]:
for i in df.values: 
    print(i[2])

81d6c1c36fe913d9103870acf4a95eadc7bc2fd1
d61e33a1a2597cb37a271110b96d23b5cfe1e678
79737a255c57c75d65d9ac6a9c9d2b1a483f1df8
e7e5c5e26cad77f7ef7f8bb01f139fc78c630f47
3531059b9d9a012e6b003af7487f1739d3cfd77f
169df4543bed65ae5973559ad88356a460add3e3
0a4cf145d58e1b1c08ff4d57dc6c28180720a473
514b09d7fa1f83a73d89a93a15f02f8473b13461
c933398576ff9a526669476c53760182bec7111f
05c175de544da972fadb5d45c7ed045e6b5aa8fd
9f642d0ca0ced7364e4ad59516895556f5903599
22e7a21ecf039b934a542173dbd9652de7c44bec
41b862b17ef34760e3f5c87745a8fba3967ad3a7
ab52ab8895c5f21eba492ece28ae52aa99534179
6e948385c2da62417db26f9a55d2fd0209ea4bb7
dd226ccf9269af818c948989ece0a0a3e3f1b175
23ee2e3011d96e21abe20508f0f53c9ee3130799
cab90fc0b5e947a1759f74cb8213429b0ec7734f
c3132e113a2a34f5edefbd982aa8738a684777ff
dd32419f4af02a38be8dca126cc29dcfde5166aa
f8a690ea2c88a17c3cc93b79a4890c927ae61166
4ab4e7568900e798c9db4d7b544b28261ecbe3e5
fa2e9266298690b5ea51e0e27788637f7153bcc3
e61d7527a36be4f9bcc6d3eac5c11dca1ad3819e
0454c571e14dfddd